In [1]:
import numpy as np
import pandas as pd

# 베이지안 모델
    
베이지안 네트워크는, belief network이다. 일반적으로 확률 변수 간의 causal relationship을 나타내는데 사용한다. 베이지안 네트워크는 parameterized 에 조건부 확률 분포를 사용한다. (Conditional Probability Distribution). 각각의 노드들은 parameterized 된다 $ P(node | Pa(node)) $ 이런식으로, $ Pa(node) $ 이건 부모 노드를 표현하는 방식이다.

- Representation
    - Directed & acyclic (DAG)
    - Probabilistic graphical model
    - Set of random variables and conditional dependency
    
    
### Summary

Bayisan Model : 방향성이 있는 그래프이자 조건부 확률 분포를 따른다. 각각 조건부 확률 분포는 $ P(node | parents(node)) $ 의 형태이며, $ parents(node)$ 각 노드의 부모역할을 한다.


![Alt text](images/2/student_full_param.png)

본 문서에서는 pgmpy를 이용하여 network structure를 정의하고, CPD를 이용하여 개별 노드를 정의해본다.

In [2]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

In [3]:
# define model sturcture
model = BayesianModel([('D','G'),('I','G'),('G','L'),('I','S')])

In [6]:
# define individual cpd

cpd_d = TabularCPD(variable='D', variable_card=2, values=[[0.6,0.4]])
cpd_i = TabularCPD(variable='I', variable_card=2, values=[[0.7,0.3]])
cpd_g = TabularCPD(variable='G', variable_card=3, 
                   values=[[0.3, 0.05, 0.9,  0.5],
                           [0.4, 0.25, 0.08, 0.3],
                           [0.3, 0.7,  0.02, 0.2]],
                  evidence=['I', 'D'],
                  evidence_card=[2, 2])

cpd_l = TabularCPD(variable='L', variable_card=2, 
                   values=[[0.1, 0.4, 0.99],
                           [0.9, 0.6, 0.01]],
                   evidence=['G'],
                   evidence_card=[3])

cpd_s = TabularCPD(variable='S', variable_card=2,
                   values=[[0.95, 0.2],
                           [0.05, 0.8]],
                   evidence=['I'],
                   evidence_card=[2])

In [8]:
# 모델에 각 확률 변수들을 연결시켜주자 = 네트워크를 구성시키자
model.add_cpds(cpd_d,cpd_g,cpd_i,cpd_l,cpd_s)

In [9]:
# 모델 스트럭쳐가 잘 구성되었는지를 확인해본다. 일반적으로 summation이 1인지를 보면 안다
model.check_model()

True

In [11]:
# 모델 스트럭쳐와 각 노드들을 확인하는 법
model.get_cpds()

[<TabularCPD representing P(D:2) at 0x1a24d150f0>,
 <TabularCPD representing P(G:3 | I:2, D:2) at 0x1a24d15160>,
 <TabularCPD representing P(I:2) at 0x1a24d150b8>,
 <TabularCPD representing P(L:2 | G:3) at 0x1a24cf0978>,
 <TabularCPD representing P(S:2 | I:2) at 0x1a24d15198>]

In [12]:
print(model.get_cpds('G'))

╒═════╤═════╤══════╤══════╤═════╕
│ I   │ I_0 │ I_0  │ I_1  │ I_1 │
├─────┼─────┼──────┼──────┼─────┤
│ D   │ D_0 │ D_1  │ D_0  │ D_1 │
├─────┼─────┼──────┼──────┼─────┤
│ G_0 │ 0.3 │ 0.05 │ 0.9  │ 0.5 │
├─────┼─────┼──────┼──────┼─────┤
│ G_1 │ 0.4 │ 0.25 │ 0.08 │ 0.3 │
├─────┼─────┼──────┼──────┼─────┤
│ G_2 │ 0.3 │ 0.7  │ 0.02 │ 0.2 │
╘═════╧═════╧══════╧══════╧═════╛


In [13]:
model.get_cardinality('G')

3

### 베이지안 네트워크의 독립 (Independencies)

구조속에서 독립관계를 파악하는 것은 베이지안 네트워크에서 매우 중요하다. 네트워크는 2가지 타입으로 구별할 수 있다.

1. __Local Independencies__ :
베이지안 네트워크에서 어떤 확률 변수들은 부모노드들이 given condition으로 주어질때 Non-Descendents 한 관계인 노드들과 조건부 독립 관계를 가지게 된다.

$$ (X \perp NonDesc(X) | Pa(X) $$

2. __Global Independencies__ : 
오직 두개의 노드가 직접적으로 연결된 경우, 그들은 global independencies 를 보장 받는다.

![Alt text](images/2/two_nodes.png)

우리의 첫 예시인 student 케이스를 다시보자
![Alt text](images/2/student_full_param.png)

$ difficulty \rightarrow grade $ 이 경우를 확인해보자, 만약 우리가 difficulty를 높이면 등급이 올라갈 확률은 줄어든다. 
<br>
$ SAT \leftarrow Intel $ 이 경우는 어떠한가, 우리가 만약에 SAT의 점수가 높다는 것을 관찰할 확률을 높이게 된다면, 이는 지능이 좋다는 확률을 높일 것이다. 이 지능은 grade의 확률에도 연결이 될 것이다.
<br>

이처럼 확률변수가 관측에 따라서 사후 확률값이 변하게 될 경우,이는 모든 베이지안 네트워크에 영향력을 끼치게 된다.

### Flow of Probabilistic Influence

![Alt text](images/2/three_nodes.png)

이제 우리는 $A$와 $C$사이에 전파되는 영향력의 플로우를 확인해 보려고 한다.

1. __Causal__ : 일반적으로, 우리가 확률 변수 $A$의 관측이 이루어졌다면, 이것은 $B$에 영향을 끼칠 것이며, $B$는 $C$에 영향을 끼칠 것이다. 
    - __cascading__ : 반면, B가 관측되었다고 해보자. 우리는 B의 상태를 파악할 것이다. 그렇다면 B는 이미 결정된 값이다. 따라서 A의 변화에 따라서 B는 영향을 받지 않는다. B가 고정되었다는 의미는 C에게도 영향력을 끼치지 않는 다는 소리이다. $ (A \perp C | B)$
    
2. __Evidential__ : Causal의 역의 관계이다. 


3. __Common Evidence__ (V-structure) : 두개의 부모노드가 하나의 자식노드에게 영향을 끼치는 케이스다. 우리한번 다시 과목의 어려움, 성적, 지능의 관계를 살펴보자 $ D \rightarrow G \leftarrow I $. 만약 코스가 어려울 확률이 올라간다고 했을때, 성적이 잘나올 확률을 감소한다. 하지만 이 경우 학생의 지능에는 영향을 끼치지 않는다. 그러나 우리가 성적이 잘나왔다는 것이 관측되었다고 해보자. 그렇다면 우리는 이제 코스가 어려울 확률이 올라간다고 했을때, 이 학생의 지능이 좋다고 나올 확률이 올라감을 알 수 있다. 독립이였던 $ (D \perp I) $ 과목의 어려움과 지능이, $ ( A \not\perp C | B) $ 성적의 관측에 의해서 관계가 형성된 경우이다.

4. __Common Cause__ (commen parents): 하나의 부모노드(B)와 두개의 자식노드(A,C)가 있는 케이스이다. B가 관측된 순간 A와 C는 독립관계가 형성된다. 왜냐면 오직 B에 대해서 dependent 해지기 때문이다. $ ( A \perp C | B) $. 

In [16]:
# 확률변수의 local_indepdendencies를 보자
model.local_independencies('G')

(G _|_ S, L | D, I)

In [17]:
# 이제 모델의 모든 변수를 조회한다면.
model.local_independencies(['D', 'I', 'S', 'G', 'L'])

(D _|_ G, S, L, I)
(I _|_ G, D, S, L)
(S _|_ G, D, L | I)
(G _|_ S, L | D, I)
(L _|_ D, S, I | G)

In [18]:
# active_trail : 두개의 확률변수가 서로 영향을 끼치는 관계 인지를 확인하는 것입니다.
model.active_trail_nodes('D')

{'D': {'D', 'G', 'L'}}

In [20]:
# observed : 특정 확률 변수가 관측되어서 상수가 될 경우
model.active_trail_nodes('D', observed='G')

{'D': {'D', 'I', 'S'}}

### Representing Joint Distribution

이제 Joint Distribution을 확인해봅시다.